# Tutoriel 4


[Télécharger le tutoriel](04_tutoriel.ipynb)

# Se familiariser avec la discrétisation spatiale

Pour résoudre une équation en 1D (par exemple, une équation de diffusion), il est nécessaire de discrétiser l'espace, c'est-à-dire de diviser le domaine spatial de modélisation en intervalles avec un ensemble de points où la solution sera calculée. Par exemple, si notre domaine est l'intervalle $[0, 1]$, nous définissons $x_0 = 0$, $x_1 = 0.1$, $x_2 = 0.2$, ..., $x_{10} = 1$, avec une longueur d'intervalle $dx = 0.1$.

Dans l'example suivant, nous discrétisons l'intervalle $[a,b]$ en $nx$ morceaux: $x_0$ -- $x_1$ -- $x_2$ -- ... -- ... -- ... -- ... -- $x_{nx-2}$ -- $x_{nx-1}$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

a = 0
b = 1
nx = 101                            
x  = np.linspace(a, b, nx)  
dx = (b-a)/(nx-1)

print('Le vecteur des coordonééer de la discretisation est :', x)
print('Sa longueur est :', len(x))
print('La valeur de dx est :', dx)

Maintenant que notre domaine est discrétisé, nous pouvons initialiser une fonction (comme une concentration ou une température) sur ce domaine. Par exemple, nous pouvons initialiser une température égale à 2 partout :

In [ ]:
T = np.ones(nx)*2

Nous pouvons vérifier en traçant la fonction T(x) que nous l'avons bien initialisée :

In [ ]:

plt.plot(x, T) , plt.xlabel('x') , plt.ylabel('T')

Il est possible de modifier la valeur de T en un point donné en utilisant la syntaxe suivante :

In [ ]:
T[0]  = 0             # modification de la valeur de la première composante (cad quand x=a)
T[-1] = 5             # modification de la valeur de la dernière composante (cad quand x=b)
T[int(nx/5):int(2*nx/5)] = -1 # modification de la valeur de la composante entre le premier et le deuxième cinquième du domaine

Affichons à nouveau la fonction $T$ pour voir les effets des changements :

In [ ]:
plt.plot(x, T) , plt.xlabel('x') , plt.ylabel('T')

Nous avons vu au point précédent comment changer la valeur de T à un point donné. Souvent, il est demandé de le faire à un point physique. Par exemple, on souhaite donner à la température T la valeur $6$ au point $x_P = 0.4$. Cependant, il n'est pas correct de faire `T[0.4] = 6`, car `T` est un vecteur de $n_x$ éléments et il ne prend que des indices entiers comme arguments. 

Il faut donc trouver l'indice correspondant à $x_P = 0.4$ (ou celui qui s'en approche le plus), car il n'est pas garanti que $x_P = 0.4$ soit atteint dans notre discrétisation $x_0$, $x_1$, $x_2$, ..., $x_{n_x - 2}$, $x_{n_x - 1}$. Pour cela, nous devons diviser la position du point $x_P$ par la longueur de l'intervalle $dx$ pour obtenir sa position relative au début de l'intervalle. Ensuite, nous prenons la partie entière de ce résultat pour nous assurer que l'indice obtenu est un entier : `ixp = int((x_P - a) / dx)`. 

Voici le code complet ainsi qu'un graphique pour vérifier que la valeur a bien été changée :

In [ ]:
Txp = 6   
xp = 0.6
ixp = int((xp-a)/dx) # indice de la composante la plus proche de xp
T[ixp] = Txp         # modification de la valeur de la composante la plus proche de xp
plt.plot(x, T) , plt.xlabel('x') , plt.ylabel('T') # affichage du vecteur T
plt.scatter(xp, Txp, color='red') # affichage du point rouge pour le nouveau point

En utilisant la même stratégie, nous pouvons appliquer cela sur un intervalle (ici, nous forçons la température à valoir 1 entre $x$ = 0.1 et $x$ = 0.9) :

In [ ]:
Txp = 1  
xl = 0.1
xr = 0.9
ixl = int((xl-a)/dx) 
ixr = int((xr-a)/dx) 
T[ixl:ixr] = Txp     
plt.plot(x, T) , plt.xlabel('x') , plt.ylabel('T') # affichage du vecteur T

Notons qu'il est possible (et parfois préférable) d'obtenir le même résultat que `T[ixl,ixr]` avec avec `T[(xl<x)&(x<xr)] = Txp`. Avec cette derniere commande et `x` étant le vecteur de coordonnés défini ci-dessus, `(xl<x)&(x<xr)` trouve les indices du vecteur dont les coordonées sont compris entre `xl` et `xr`, il est ainsi plus facile d'attribuer une valeur choisie `Txp` aux éléments du vecteur `T` qui satisfassent cette condition. Ainsi la comande suivante est équivalente à celle du code précédent:

In [ ]:
T[(xl<x)&(x<xr)] = Txp

Il est possible de tester si la température T au point `vc` est égale à `va` avec la commande `T[int((xc - a)/dx)] == va` :

In [ ]:
xc = 0.3 ; va = 1 ; print('La température en x=' + str(xc) + ' est equal à ' + str(va) + ' :', T[int((xc-a)/dx)]==va)
xc = 0.0 ; va = 3 ; print('La température en x=' + str(xc) + ' est equal à ' + str(va) + ' :', T[int((xc-a)/dx)]==va)
xc = 0.0 ; va = 0 ; print('La température en x=' + str(xc) + ' est equal à ' + str(va) + ' :', T[int((xc-a)/dx)]==va)
xc = 0.8 ; va = 0 ; print('La température en x=' + str(xc) + ' est equal à ' + str(va) + ' :', T[int((xc-a)/dx)]==va)

Il sera parfois nécessaire de calculer des valeurs entre les nœuds de discrétisation. Cela peut se faire en faisant une moyenne entre deux points successifs. Notons qu'en faisant cela, nous avons perdu une cellule ; le vecteur est maintenant de dimension $n_x - 1$.

In [ ]:
Tmid = (T[1:]+T[:-1])/2 # calcul de la valeur de la température au milieu de chaque intervallele (taille nx-1)
xmid = (x[1:]+x[:-1])/2 # calcul de la valeur de la position au milieu de chaque intervallele (taille nx-1)

print("Coordonnées des points au milieu de chaque intervalle :")
print(xmid)
print("Valeurs de la température au milieu de chaque intervalle :")
print(Tmid)

# Se familiariser avec l'approximation d'un dérivée spatiale

Pour résoudre un modèle (p.e. de diffusion), il nous faudra trés souvent dans ces modèles appoximer des dérivées, c.a.d. de facçon numérique (pas de façon exacte/analytique). Pour illustrer cela, nous définisons un vecteur $F$ avec la fonction sinus, sa dérivée "exacte" définit avec la fonction cosinus, et sa dérivée "approchée" défini au moyen d'une différence finie (approximation que nous utiliserons à mainte reprise dans ce cours). Le code suivant fait tout cela, et montre que la dérivée approximative est trés proche de la dérivée exacte:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define x and dx
nx = 100
x = np.linspace(0, 2 * np.pi, nx)  # Example intervalle of x
dx = x[1] - x[0]  # Calculer dx

# Calculer F, dFdx_exact, and dFdx_appro
F = np.sin(x)
dFdx_exact = np.cos(x)
dFdx_appro = (F[1:] - F[:-1]) / dx

# Calculer points milieux (xm)
xm = (x[1:] + x[:-1]) / 2

# Tracerting
plt.plot(x, dFdx_exact, label="dF/dx exact (blue)", color="blue")
plt.plot(xm, dFdx_appro, label="dF/dx approx (red)", color="red", linestyle="--")

# Add labels, legend, and title
plt.xlabel("x")
plt.ylabel("dF/dx")
plt.title("Comparaison entre dérivées Exact et Approximative")
plt.legend()

# Afficher the tracer
plt.grid(True)
plt.show()

Notons que pour afficher les deux `dFdx_exact` et `dFdx_approx` sur un meme graphique, il a fallue calculer les points au centre des cellules `xm = (x[1:] + x[:-1]) / 2` cat la dérivée approximative est de taille `nx-1` puisque l'on perd une cellule lors de la différence finie `(F[1:] - F[:-1]) / dx`, il faut donc "plotter" `dFdx_approx` par rapport à `xm` et non `x` qui est de taille `nx`.